## **Create CNN Model and Optimize it using Keras-Tuner**

In [1]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 3.9 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
print(tf.__version__)

2.15.0


In [3]:
fashion_mnist = keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


## Scaling the pixels of the images between 0 and 1

In [5]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp) :
    model = keras.Sequential([
        keras.layers.Conv2D(                                                                        # first convolutional layer
            filters = hp.Int('CL1_filter' , min_value = 32 , max_value = 128 , step = 16),          # number of kernels
            kernel_size = hp.Choice('CL1_kernel' , values = [3,5]),                                 # size of each kernel
            activation = 'relu',
            input_shape = (28,28,1)
        ),
        keras.layers.Conv2D(                                                                        # second convolutional layer
            filters = hp.Int('CL2_filter' , min_value = 32 , max_value = 128 , step = 16),
            kernel_size = hp.Choice('CL2_kernel' , values = [3,5]),
            activation = 'relu'
        ),
        keras.layers.Flatten(),
        keras.layers.Dense(
            units = hp.Int('dense_1_units' , min_value = 32 , max_value = 128 , step = 16),
            activation = 'relu',
        ),
        keras.layers.Dense(10 , activation = 'softmax')                                             # output layer
    ])
    model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate' , values = [1e-2, 1e-3])),
                  loss = 'sparse_categorical_crossentropy',
                  metrics = ['accuracy']
                  )
    return model

In [12]:
!pip install keras-tuner --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 4.5 MB/s eta 0:00:00


In [14]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [15]:
tuner_search = RandomSearch(build_model,
                            objective = 'val_accuracy',
                            max_trials = 5,
                            directory = 'output',
                            project_name = 'Mnist Fashion')

In [16]:
tuner_search.search(train_images, train_labels, epochs = 3, validation_split = 0.1)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.9071666598320007

Best val_accuracy So Far: 0.9153333306312561
Total elapsed time: 00h 03m 33s


In [21]:
model = tuner_search.get_best_models(num_models = 1)[0]

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 112)       89712     
                                                                 
 flatten (Flatten)           (None, 54208)             0         
                                                                 
 dense (Dense)               (None, 96)                5204064   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 5295066 (20.20 MB)
Trainable params: 5295066 (20.20 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
model.fit(train_images, train_labels, epochs = 10, validation_split = 0.1, initial_epoch = 3)

Epoch 4/10
1688/1688 [==============================] - 12s 6ms/step - loss: 0.1121 - accuracy: 0.9585 - val_loss: 0.2543 - val_accuracy: 0.9170
Epoch 5/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0757 - accuracy: 0.9722 - val_loss: 0.2927 - val_accuracy: 0.9130
Epoch 6/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0508 - accuracy: 0.9811 - val_loss: 0.3757 - val_accuracy: 0.9110
Epoch 7/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0380 - accuracy: 0.9864 - val_loss: 0.4361 - val_accuracy: 0.9107
Epoch 8/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0291 - accuracy: 0.9894 - val_loss: 0.4548 - val_accuracy: 0.9127
Epoch 9/10
1688/1688 [==============================] - 11s 7ms/step - loss: 0.0258 - accuracy: 0.9912 - val_loss: 0.4711 - val_accuracy: 0.9110
Epoch 10/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0190 - accuracy: 0.9935 - val_loss: 0.5385 - val_a